# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    #TODO 1: Add your code here
    column_names = ['Title', 'Year', 'Synopsis', 'Review', 'Original Language']
    
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_es = pd.read_csv("data/movie_reviews_sp.csv")

    df_eng['Original Language'] = 'EN'
    df_fr['Original Language'] = 'FR'
    df_es['Original Language'] = 'ES'

    df_eng.columns = column_names
    df_fr.columns = column_names
    df_es.columns = column_names

    df = pd.concat([df_eng, df_fr, df_es]).reset_index(drop=True)
    return df

df = preprocess_data()

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",EN
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",FR
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",EN
10,La La Land,2016,Cette comédie musicale raconte l'histoire d'un...,"""La La Land est un film absolument magnifique ...",FR
29,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",ES
26,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi...",ES
21,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",ES
15,Le Dîner de Cons,1998,Le film suit l'histoire d'un groupe d'amis ric...,"""Je n'ai pas aimé ce film du tout. Le concept ...",FR
25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",ES
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",EN


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = 'Helsinki-NLP/opus-mt-fr-en'
es_en_model_name = 'Helsinki-NLP/opus-mt-es-en'
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors='pt')

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [5]:
# Test out translation
fr_review = df.loc[df['Original Language']=='FR']
print(translate(fr_review.iloc[0]['Title'], fr_en_model, fr_en_tokenizer))
print(translate(fr_review.iloc[0]['Synopsis'], fr_en_model, fr_en_tokenizer))
print(translate(fr_review.iloc[0]['Review'], fr_en_model, fr_en_tokenizer))

The Land
This musical tells the story of a budding actress and jazz pianist who fall in love in Los Angeles. The film was hailed for its enchanting music, dazzling dance scenes and exceptional performances.
"The Land is an absolutely beautiful film with songs that stay in the head for days. The actors are incredible and their alchemy is palpable. The dance scenes are absolutely dazzling and the story is touching and authentic."


In [6]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df.loc[df['Original Language']=='FR']['Review']
fr_reviews_en = fr_reviews.apply(lambda text: translate(text, fr_en_model, fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df.loc[df['Original Language']=='FR']['Synopsis']
fr_synopsis_en = fr_synopsis.apply(lambda text: translate(text, fr_en_model, fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df.loc[df['Original Language']=='ES']['Review']
es_reviews_en = es_reviews.apply(lambda text: translate(text, es_en_model, es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df['Original Language']=='ES']['Synopsis']
es_synopsis_en = es_synopsis.apply(lambda text: translate(text, es_en_model, es_en_tokenizer))

In [7]:
# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[fr_reviews.index, 'Review'] = fr_reviews_en
df.loc[fr_reviews.index, 'Synopsis'] = fr_synopsis_en
df.loc[es_reviews.index, 'Review'] = es_reviews_en
df.loc[es_reviews.index, 'Synopsis'] = es_synopsis_en

In [8]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",FR
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",FR
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",ES
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",EN
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",ES
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",FR
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",ES
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",FR
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",FR
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",ES


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [9]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
sentiment_classifier = pipeline(model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    output = classifier(text)
    sentiment = output[0]['label'].title()
    
    return sentiment

In [10]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df['Sentiment'] = df['Review'].apply(lambda text: analyze_sentiment(text, sentiment_classifier))

In [11]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",FR,Negative
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",EN,Positive
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",EN,Positive
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",EN,Negative
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",ES,Positive
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",FR,Positive
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",EN,Negative
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",ES,Positive
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",FR,Negative
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",FR,Positive


In [12]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)